In [1]:
import numpy as np 
from collections import deque
import numba 

In [2]:
rng = np.random.default_rng(12345)
np.random.seed(12345)

In [4]:
%timeit rng.integers(0,1000,1000)
%timeit np.random.randint(0,1000,1000)

14.8 µs ± 33 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
15.9 µs ± 110 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
%timeit np.random.randint(0, 128, 100)
%timeit (128*np.random.random(100)).astype(int)

11.5 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
4.66 µs ± 80 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
%timeit np.random.normal(0, 1, 100000)
%timeit np.random.randn(100000)

2.99 ms ± 14.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.86 ms ± 27.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
